# YOLO

Este notebook abordará a utilização e o ajuste fino (fine-tuning) da arquitetura **You Only Look Once (YOLO)**, especificamente implementada pela biblioteca **Ultralytics**. O YOLO é um detector de objetos em tempo real que reformulou a detecção ao tratar o problema como uma regressão única. O fine-tuning é uma técnica crucial no Deep Learning que permite adaptar um modelo pré-treinado em um *dataset* vasto (como COCO) para uma tarefa específica com um *dataset* menor e mais especializado, aproveitando o conhecimento hierárquico já aprendido pelo modelo base.

A biblioteca **Ultralytics YOLO** fornece uma interface de alto nível e eficiente para diversas versões do YOLO (YOLOv8, YOLOv5, etc.), tornando a inferência e o treinamento acessíveis. A filosofia é focar na usabilidade sem sacrificar a performance.

Para começar, demonstraremos a inferência básica, que é o processo de usar um modelo treinado para fazer previsões em novos dados. Utilizaremos um modelo pré-treinado em um *dataset* padrão para a detecção de objetos.

In [ ]:
# Instalação da biblioteca ultralytics. Pode ser necessário reiniciar o runtime.
!pip install ultralytics

In [ ]:
import os
import random
import glob
from IPython.display import display
from PIL import Image
from ultralytics import YOLO

import ultralytics
print(f"Ultralytics version: {ultralytics.__version__}")

## Inferência

O processo de inferência com o YOLO da Ultralytics é notavelmente simples. Basta carregar um modelo pré-treinado, como o `yolov8n.pt` (a versão *nano* do YOLOv8, que oferece um bom equilíbrio entre velocidade e precisão) e chamar o método `predict` com a fonte de dados desejada.

### Inferência em Imagens

A inferência em uma imagem envolve o carregamento do arquivo e a obtenção das caixas delimitadoras (*bounding boxes*), classes e pontuações de confiança para os objetos detectados.

In [ ]:
model = YOLO('data/yolov8n.pt')

Executando a inferência em uma imagem de exemplo (pode ser um URL ou caminho local). O Ultralytics baixa automaticamente os pesos do modelo na primeira execução. O modo `save=True` salva a imagem com as detecções desenhadas. Usaremos uma imagem de exemplo que o próprio Ultralytics pode acessar.

É importante notar que o argumento `source` aceita caminhos para arquivos locais, URLs e até mesmo index de câmeras.

In [ ]:
results = model.predict(source='https://ultralytics.com/images/bus.jpg', save=False, conf=0.25)

In [ ]:
for r in results:
    r.show()

In [ ]:
# 'results' é uma lista de objetos Results, um para cada fonte de entrada (no caso, apenas um).
for r in results:
    print(f"Número de objetos detectados: {len(r.boxes)}")
    for box in r.boxes:
        print(f"Coordenadas: {box.xyxy.cpu().numpy()[0]} - Confiança: {box.conf.cpu().numpy()[0]:.2f} - Classe: {box.cls.cpu().numpy()[0]:.0f}")

## Fine-tuning

O **Fine-tuning** é a parte central da transferência de aprendizado (*Transfer Learning*). A premissa é que os pesos de um modelo treinado em um *dataset* genérico (como o COCO, que contém 80 classes gerais) já capturaram características de baixo nível (bordas, texturas, formas) e de alto nível (partes de objetos) que são úteis para tarefas correlatas.

Ao realizar o *fine-tuning*, carregamos esses pesos pré-treinados e continuamos o treinamento em nosso *dataset* específico, geralmente com uma **taxa de aprendizado** (*learning rate*) muito menor, para evitar a destruição das representações úteis aprendidas previamente.

### Carregamento do Dataset no Padrão YOLO

O Ultralytics YOLO espera que o *dataset* esteja em um formato específico. O padrão de estrutura de diretórios é importante:

```

dataset_name/
│
├── images/
│   ├── train/
│   ├── val/
│   └── test/        # opcional
│
└── labels/
    ├── train/
    ├── val/
    └── test/        # opcional
```

O arquivo de configuração `YAML` deve conter os caminhos para as pastas, o número de classes (`nc`) e os nomes das classes (`names`).

Utilizaremos um pequeno *dataset* público para fins didáticos. O dataset de **frutas (Fruits-360)**, adaptado para o padrão YOLO, é um bom exemplo.

In [ ]:
DATASET_DIR = "data/fruit-detection/Fruits-detection/"

In [ ]:
!curl -L -o data/fruit-detection.zip https://www.kaggle.com/api/v1/datasets/download/lakshaytyagi01/fruit-detection
!unzip -q data/fruit-detection.zip -d data/fruit-detection

In [ ]:
# names:
# - Apple
# - Banana
# - Grape
# - Orange
# - Pineapple
# - Watermelon
# nc: 6
# test: test/images
# train: train/images
# val: valid/images

###  Execução do Fine-tuning

O fine-tuning é realizado utilizando o comando `model.train()`. É essencial carregar um modelo pré-treinado (como o `yolov8n.pt`) para inicializar os pesos.

Os principais hiperparâmetros a serem configurados são:
* `data`: O caminho para o arquivo YAML de configuração do *dataset*.
* `epochs`: O número de épocas de treinamento. Para fine-tuning, geralmente são necessárias poucas épocas (10-50).
* `imgsz`: O tamanho da imagem de entrada.
* `batch`: O tamanho do *batch*. Deve ser ajustado à memória da GPU.

O processo de treinamento irá ajustar os pesos do modelo pré-treinado para as classes e características específicas do novo *dataset*.

In [ ]:
# Carrega o modelo pré-treinado novamente
model = YOLO('yolov8n.pt')

In [ ]:
results = model.train(
    data=os.path.join(DATASET_DIR, "data.yaml"),
    epochs=1,
    imgsz=640,
    batch=32,
    name='yolov8n_fruits_finetune'
)

print("Fine-tuning concluído. O modelo treinado (best.pt) foi salvo em 'runs/detect/yolov8n_fruits_finetune/weights'.")

## Análise e Inferência com o Modelo Ajustado

Após o treinamento, o modelo com o melhor desempenho (*best.pt*) é salvo. É fundamental avaliar o modelo ajustado para garantir que ele aprendeu a detectar as novas classes de interesse de forma eficaz.

O principal indicador de desempenho em detecção de objetos é o **mAP (mean Average Precision)**, que agrega as curvas de Precision-Recall para cada classe.

$$
mAP = \frac{1}{N_{cl}} \sum_{i=1}^{N_{cl}} AP_i
$$

Onde $N_{cl}$ é o número de classes e $AP_i$ é a Precisão Média para a classe $i$. O Ultralytics geralmente reporta o $mAP50$ (mAP com IoU - *Intersection over Union* - de 0.5) e o $mAP50-95$ (média do mAP em vários *thresholds* de IoU de 0.5 a 0.95).

In [ ]:
# Carregando o modelo ajustado (best.pt)
tuned_model = YOLO('runs/detect/yolov8n_fruits_finetune/weights/best.pt')

In [ ]:
n = 4
val_dir = os.path.join(DATASET_DIR, "valid/images")
image_paths = glob.glob(val_dir + "/*.jpg")
sample_paths = random.sample(image_paths, n)

for img_path in sample_paths:
    results = model.predict(source=img_path, conf=0.25, verbose=False)
    annotated_array = results[0].plot()  # BGR
    annotated_img = Image.fromarray(annotated_array[:, :, ::-1])  # agora RGB
    
    print(f"Arquivo: {img_path}")
    display(annotated_img)